In [ ]:
#| label: filtFig4jn

# Prepare Python environment

import scipy.io as sio
from pathlib import Path

from contextlib import contextmanager
import sys, os
from pathlib import Path

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

import os
from pathlib import Path

def find_myst_yml_directories(start_dir=None):
    """
    Recursively search for directories containing myst.yml file.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        list: List of full paths to directories containing myst.yml
    """
    if start_dir is None:
        start_dir = Path.cwd()
    else:
        start_dir = Path(start_dir)
    
    myst_dirs = []
    
    def _search_directory(current_dir):
        # Check if myst.yml exists in current directory
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            myst_dirs.append(str(current_dir.resolve()))
            # Don't search subdirectories if we found myst.yml here
            return
        
        # Recursively search all subdirectories
        for item in current_dir.iterdir():
            if item.is_dir():
                try:
                    _search_directory(item)
                except (PermissionError, OSError):
                    # Skip directories we can't access
                    continue
    
    _search_directory(start_dir)
    return myst_dirs

def find_myst_yml_directories_upwards(start_dir=None):
    """
    Search for myst.yml in current directory, if not found go to parent and repeat.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        str or None: Full path of directory containing myst.yml, or None if not found
    """
    if start_dir is None:
        current_dir = Path.cwd()
    else:
        current_dir = Path(start_dir)
    
    # Keep going up until we reach the filesystem root
    while current_dir != current_dir.parent:  # Stop at root
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            return str(current_dir.resolve())
        
        # Move to parent directory
        current_dir = current_dir.parent
    
    return None

with suppress_stdout():
    repo_path = Path(find_myst_yml_directories_upwards())
    print(repo_path)
    data_req_path = repo_path / "binder" / "data_requirement.json"
    data_path = repo_path / "data"
    dataset_path = data_path / "qmrlab-mooc"

data_dir = dataset_path / "04-B1-03-Filtering" / "04-B1-03-Filtering"
data_file = "b1filt_fig1.mat"

#Load either archived or generated plot variables
mat_contents = sio.loadmat(data_dir / data_file)

b1_func = mat_contents["b1_func"][0]
gauss_b1_1d = mat_contents["gauss_b1_1d"]
median_b1_1d = mat_contents["median_b1_1d"]
spline_b1_1d = mat_contents["spline_b1_1d"]
vox_range = mat_contents["vox_range"][0]

## Plot
# Module imports

import matplotlib.pyplot as plt
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

# PYTHON CODE

init_notebook_mode(connected=True)
# The polling here is to ensure that plotly.js has already been loaded before
# setting display alignment in order to avoid a race condition.

position = np.arange(1,len(b1_func)+1)

b1_data = [dict(
            visible = False,
            x = position,
            y = b1_func,
            name = "True signal",
            hoverinfo = "y") for ii in vox_range]

b1_data[5]['visible'] = True


data_gauss = [dict(
        visible = False,
        x = position,
        y = list(gauss_b1_1d[ii-1]),
        name = "Gaussian filtered",
        hoverinfo = "y") for ii in vox_range]

data_gauss[5]['visible'] = True

data_median = [dict(
        visible = False,
        x = position,
        y = list(median_b1_1d[ii-1]),
        name = "Median filtered",
        hoverinfo = "y") for ii in vox_range]

data_median[5]['visible'] = 'legendonly'

data_spline = [dict(
        visible = False,
        x = position,
        y = list(spline_b1_1d[ii-1]),
        name = "Spline filtered",
        hoverinfo = "y") for ii in vox_range]

data_spline[5]['visible'] = 'legendonly'

data = b1_data + data_gauss + data_median + data_spline

# Create and add slider
steps = []
      
for i in vox_range:
    step = dict(
        method="update",
        args=[{"visible": [False] * len(data_gauss)},],  # layout attribute
        label = str(i)
    )
    step["args"][0]["visible"][i-1] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=5,
    currentvalue={"prefix": "Filter width or spline/polynomial order: "},
    pad={"t": 50},
    steps=steps
)]

layout = go.Layout(
    width=750,
    height=750,
    margin=go.layout.Margin(
        l=100,
        r=80,
        b=100,
        t=130,
    ),
    annotations=[
        dict(
            x=-0.15,
            y=0.50,
            showarrow=False,
            text='Signal',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        showgrid=True,
        gridcolor='rgb(169,169,169)',
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='rgb(169,169,169)',
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.25,
        y=1.3,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ),
    sliders=sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'fig1.html', config = config)
